# CS 418: Introduction to Data Science
**Final Project - Fall 2020**

#### Instructions
**This assignment is due Monday, November 30, at 11:59PM (Central Time).
For this assignment, you must work in teams of three students. Each member of the
team must be assigned one of three roles (project manager, scribe, or timekeeper) and
everyone must switch roles in every project.
Deliverables for this assignment (see Deliverables section below) must be submitted
on Blackboard by the project manager. Only one submission per team is required.
Additionally, every member of the team must submit a self- and peer-evaluation form.
No late submissions will be accepted for the final project.
Offering or receiving any kind of unauthorized or unacknowledged assistance is a
violation of the University’s academic integrity policies, will result in a grade of zero for
the assignment, and will be subject to disciplinary action.**

#### Project Description
**For this project, you will implement every step of the data science pipeline for a problem of your
choice. You must perform each of the following tasks:**

In [ ]:
# Load libraries
import pandas as pd
import numpy 
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys
from sklearn.cluster import KMeans, DBSCAN
from sklearn.neighbors import KNeighborsClassifier
from lib import *
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler   
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import linear_model
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn import tree
from sklearn import metrics
import matplotlib.pyplot as plt
import plotly.figure_factory as ff

In [ ]:
# United States of America Python Dictionary to translate States,
# Districts & Territories to Two-Letter codes and vice versa.
#
# https://gist.github.com/rogerallen/1583593
#
# Dedicated to the public domain.  To the extent possible under law,
# Roger Allen has waived all copyright and related or neighboring
# rights to this code.

us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

# thank you to @kinghelix and @trevormarburger for this idea
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

In [ ]:
# Load dataset and display the first five rows
confirmed_cases = pd.read_csv('covid_confirmed_usafacts.csv')
confirmed_cases.head()

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,2435,2456,2481,2506,2529,2554,2580,2597,2617,2634
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,7596,7646,7696,7772,7849,7933,8038,8131,8199,8269
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,1123,1128,1130,1134,1137,1145,1151,1157,1160,1161
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,978,986,993,1004,1008,1011,1024,1036,1136,1142


In [ ]:
confirmed_cases['Confirmed Cases']= confirmed_cases['11/23/20']

In [ ]:
confirmed_cases.drop(confirmed_cases.iloc[:, -308:-1], inplace = True, axis = 1) 

In [ ]:
confirmed_cases

,countyFIPS,County Name,State,stateFIPS,Confirmed Cases
0,0,Statewide Unallocated,AL,1,0
1,1001,Autauga County,AL,1,2634
2,1003,Baldwin County,AL,1,8269
3,1005,Barbour County,AL,1,1161
4,1007,Bibb County,AL,1,1142
...,...,...,...,...,...
3190,56037,Sweetwater County,WY,56,1548
3191,56039,Teton County,WY,56,1433
3192,56041,Uinta County,WY,56,950
3193,56043,Washakie County,WY,56,325


In [ ]:
#confirmed_cases.loc[confirmed_cases['County Name'] == 'Cook County']

In [ ]:
deaths = pd.read_csv('covid_deaths_usafacts.csv')
deaths.head()

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,36,36,36,37,37,39,39,39,39,39
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,84,84,84,84,84,84,84,84,84,84
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,9,9,9,9,10,10,10,10,10,10
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,17,17,17,17,17,17,17,17,17,17


In [ ]:
deaths['Deaths']= deaths['11/23/20']
deaths.drop(deaths.iloc[:, -308:-1], inplace = True, axis = 1)
deaths

,countyFIPS,County Name,State,stateFIPS,Deaths
0,0,Statewide Unallocated,AL,1,0
1,1001,Autauga County,AL,1,39
2,1003,Baldwin County,AL,1,84
3,1005,Barbour County,AL,1,10
4,1007,Bibb County,AL,1,17
...,...,...,...,...,...
3190,56037,Sweetwater County,WY,56,6
3191,56039,Teton County,WY,56,2
3192,56041,Uinta County,WY,56,4
3193,56043,Washakie County,WY,56,7


In [ ]:
population = pd.read_csv('covid_county_population_usafacts.csv')
population

,countyFIPS,County Name,State,population
0,0,Statewide Unallocated,AL,0
1,1001,Autauga County,AL,55869
2,1003,Baldwin County,AL,223234
3,1005,Barbour County,AL,24686
4,1007,Bibb County,AL,22394
...,...,...,...,...
3190,56037,Sweetwater County,WY,42343
3191,56039,Teton County,WY,23464
3192,56041,Uinta County,WY,20226
3193,56043,Washakie County,WY,7805


In [ ]:
population[population['County Name'] == 'Dona Ana County']

,countyFIPS,County Name,State,population


In [ ]:
confirmed_cases[confirmed_cases['County Name'] == 'Dona Ana County']

,countyFIPS,County Name,State,stateFIPS,Confirmed Cases
1835,35013,Dona Ana County,NM,35,12274


In [ ]:
# removes rows that didn't have a population
merged_data = population.merge(confirmed_cases, how = 'inner', on=['State', 'County Name', 'countyFIPS'], sort = False)
merged_data

,countyFIPS,County Name,State,population,stateFIPS,Confirmed Cases
0,0,Statewide Unallocated,AL,0,1,0
1,1001,Autauga County,AL,55869,1,2634
2,1003,Baldwin County,AL,223234,1,8269
3,1005,Barbour County,AL,24686,1,1161
4,1007,Bibb County,AL,22394,1,1142
...,...,...,...,...,...,...
3188,56037,Sweetwater County,WY,42343,56,1548
3189,56039,Teton County,WY,23464,56,1433
3190,56041,Uinta County,WY,20226,56,950
3191,56043,Washakie County,WY,7805,56,325


In [ ]:
# removes rows that didn't have a population
merged_data = merged_data.merge(deaths, how = 'inner', on=['State', 'County Name', 'countyFIPS', 'stateFIPS'], sort = False)
merged_data

,countyFIPS,County Name,State,population,stateFIPS,Confirmed Cases,Deaths
0,0,Statewide Unallocated,AL,0,1,0,0
1,1001,Autauga County,AL,55869,1,2634,39
2,1003,Baldwin County,AL,223234,1,8269,84
3,1005,Barbour County,AL,24686,1,1161,10
4,1007,Bibb County,AL,22394,1,1142,17
...,...,...,...,...,...,...,...
3173,56037,Sweetwater County,WY,42343,56,1548,6
3174,56039,Teton County,WY,23464,56,1433,2
3175,56041,Uinta County,WY,20226,56,950,4
3176,56043,Washakie County,WY,7805,56,325,7


In [ ]:
merged_data = merged_data[['State', 'County Name', 'countyFIPS', 'stateFIPS', 'population', 'Confirmed Cases', 'Deaths']]
merged_data

,State,County Name,countyFIPS,stateFIPS,population,Confirmed Cases,Deaths
0,AL,Statewide Unallocated,0,1,0,0,0
1,AL,Autauga County,1001,1,55869,2634,39
2,AL,Baldwin County,1003,1,223234,8269,84
3,AL,Barbour County,1005,1,24686,1161,10
4,AL,Bibb County,1007,1,22394,1142,17
...,...,...,...,...,...,...,...
3173,WY,Sweetwater County,56037,56,42343,1548,6
3174,WY,Teton County,56039,56,23464,1433,2
3175,WY,Uinta County,56041,56,20226,950,4
3176,WY,Washakie County,56043,56,7805,325,7


In [ ]:
merged_data[merged_data['County Name'] == 'Cook County']

,State,County Name,countyFIPS,stateFIPS,population,Confirmed Cases,Deaths
434,GA,Cook County,13075,13,17270,726,16
624,IL,Cook County,17031,17,5150233,284217,6139
1353,MN,Cook County,27031,27,5463,58,0


In [ ]:
merged_data.rename(columns = {'County Name':'County', 'population':'Population'}, inplace = True)
merged_data['State'] = merged_data['State'].map(abbrev_us_state)
merged_data

,State,County,countyFIPS,stateFIPS,Population,Confirmed Cases,Deaths
0,Alabama,Statewide Unallocated,0,1,0,0,0
1,Alabama,Autauga County,1001,1,55869,2634,39
2,Alabama,Baldwin County,1003,1,223234,8269,84
3,Alabama,Barbour County,1005,1,24686,1161,10
4,Alabama,Bibb County,1007,1,22394,1142,17
...,...,...,...,...,...,...,...
3173,Wyoming,Sweetwater County,56037,56,42343,1548,6
3174,Wyoming,Teton County,56039,56,23464,1433,2
3175,Wyoming,Uinta County,56041,56,20226,950,4
3176,Wyoming,Washakie County,56043,56,7805,325,7
